In [6]:
import torch 
import torch.nn as nn
import torch.optim as optim  



In [4]:
print(torch.__version__)

2.6.0+cpu


In [12]:
def make_batch():
    input_batch=[]
    target_batch=[]
    for sen in sentences:
        word=sen.split()
        input=[word_dict[n] for n in word[:-1]]
        target=word_dict[word[-1]]
        input_batch.append(input)
        target_batch.append(target)

    return input_batch,target_batch


class NNLM(nn.Module):
    def __init__(self):
        super(NNLM,self).__init__()
        self.C=nn.Embedding(n_class,m)
        self.H=nn.Linear(n_step*m,n_hidden,bias=False)
        self.d=nn.Parameter(torch.ones(n_hidden))
        self.U=nn.Linear(n_hidden,n_class,bias=False)
        self.W=nn.Linear(n_step*m,n_class,bias=False)
        self.b=nn.Parameter(torch.ones(n_class))

    def forward(self,X):
        X=self.C(X)
        X=X.view(-1,n_step*m)
        tanh=torch.tanh(self.d+self.H(X))
        output=self.b+self.W(X)+self.U(tanh)
        return output

if __name__=='__main__':
    n_step=2
    n_hidden=2
    m=2
    sentences = ["i like dogs", "i love coffee", "i hate milk"]

    word_list=" ".join(sentences).split()
    word_list=list(set(word_list))
    word_dict = {w: i for i, w in enumerate(word_list)}
    number_dict = {i: w for i, w in enumerate(word_list)}
    n_class=len(word_dict)
    model=NNLM()
    criterion=nn.CrossEntropyLoss()
    optimizer=optim.Adam(model.parameters(),lr=0.001)
    input_batch,target_batch=make_batch()
    input_batch=torch.LongTensor(input_batch)
    target_batch=torch.LongTensor(target_batch)
    for i in range(5000):
        optimizer.zero_grad()
        output=model(input_batch)
        loss=criterion(output,target_batch)
        if(i+1)%1000==0:
            print('Epoch:', '%04d' % (i + 1), 'cost =', '{:.6f}'.format(loss))
        loss.backward()
        optimizer.step()

    predict=model(input_batch).data.max(1,keepdim=True)[1]
    print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])
        
        

Epoch: 1000 cost = 0.114707
Epoch: 2000 cost = 0.013125
Epoch: 3000 cost = 0.004238
Epoch: 4000 cost = 0.001832
Epoch: 5000 cost = 0.000900
[['i', 'like'], ['i', 'love'], ['i', 'hate']] -> ['dogs', 'coffee', 'milk']
